# Quickstart Guide for Strands Agents

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.


## Creating Your First Agent

Lets get an overview of the agentic components needed.

### 1. Create a simple Agent:

This will create an agent with the default model provider, [Amazon Bedrock](https://aws.amazon.com/bedrock/), and the default model, Claude 4 Sonnet, in the region of your AWS setup. While the agent runs in the same local environment as it is being invoked, Amazon Bedrock models will run in an AWS account and your agent will invoke the model in the cloud account. The architecture looks as following:

<div style="text-align:center">
    <img src="images/simple_agent.png" width="75%" />
</div>

In [1]:
import logging
import sys

# Patch rich Console before importing strands tools library
from rich.console import Console
_orig_init = Console.__init__
def _patched_init(self, *args, **kwargs):
    kwargs['force_terminal'] = True
    kwargs['force_jupyter'] = False
    _orig_init(self, *args, **kwargs)
Console.__init__ = _patched_init

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(name)s - %(message)s', stream=sys.stderr, force=True)

In [2]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

sys.path.append('../..')
from util.strands_retry import call_with_retry
from strands import Agent
# Initialize your agent
agent = Agent(
    model="us.anthropic.claude-sonnet-4-20250514-v1:0",  # Optional: Specify the model ID
    system_prompt="You are a helpful assistant that provides concise responses."
)

# Send a message to the agent
response = call_with_retry(lambda: agent("Hello! Tell me a joke."))

INFO - botocore.credentials - Found credentials from IAM Role: code-editor-CodeEditorInstanceBootstrapRole-Irz22oEDgalJ
INFO - strands.telemetry.metrics - Creating Strands MetricsClient


Why don't scientists trust atoms?

Because they make up everything!

### 2. Add Tools to the Agent:

The [strands-agents-tools](https://github.com/strands-agents/tools) repository provides some in-built tools which you can import. You can also create custom tools using the `@tool` decorator. We can create agents with built-in and custom tools. For instance, adding the built-in tool of a calculator and a custom tool for getting the weather you get the following architecture:
<div style="text-align:center">
    <img src="images/agent_with_tools.png" width="75%" />
</div>

Implementing this architecture you have the following:

In [3]:
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

agent = Agent(
    model="us.anthropic.claude-sonnet-4-20250514-v1:0",  # Optional: Specify the model ID
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.")

response = call_with_retry(lambda: agent("What is the weather today?"))
print(response)

INFO - botocore.credentials - Found credentials from IAM Role: code-editor-CodeEditorInstanceBootstrapRole-Irz22oEDgalJ



Tool #1: weather
The weather today is sunny! It's a beautiful day outside.The weather today is sunny! It's a beautiful day outside.



### Invoking tool directly

For some applications it is important to directly call the tool. For instance, you might want to debug the tool, pre-populate the agent knowledge with your customer's information or using a tool inside of another tool. In Strands you can do it using the ``tool`` method of your agent followed by the tool name

In [4]:
# Alternatively, you can invoke the tool directly like so:
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)

{'status': 'success',
 'content': [{'text': 'Result: -sin(x)'}],
 'toolUseId': 'tooluse_calculator_914113820'}


### 3. Changing the log level and format:

Strands SDK uses Python's standard `logging` module to provide visibility into its operations.

The Strands Agents SDK implements a straightforward logging approach:

1. **Module-level Loggers**: Each module in the SDK creates its own logger using logging.getLogger(__name__), following Python best practices for hierarchical logging.
2. **Root Logger**: All loggers in the SDK are children of the "strands" root logger, making it easy to configure logging for the entire SDK.
3. **Default Behavior**: By default, the SDK doesn't configure any handlers or log levels, allowing you to integrate it with your application's logging configuration.

To enable logging for the Strands Agents SDK, you can configure the **"strands"** logger. If you want to change the log level, for example during debugging, or modify the log format, you can set the logger configuration as follows:

In [5]:
import logging
from strands import Agent

# Enables Strands debug log level
logging.getLogger("strands").setLevel(logging.DEBUG) # or logging.INFO

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent(model="us.anthropic.claude-sonnet-4-20250514-v1:0")  # Optional: Specify the model ID
call_with_retry(lambda: agent("Hello!"))

DEBUG - strands.models.bedrock - config=<{'model_id': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'include_tool_result_status': 'auto'}> | initializing
INFO - botocore.credentials - Found credentials from IAM Role: code-editor-CodeEditorInstanceBootstrapRole-Irz22oEDgalJ
DEBUG - strands.models.bedrock - region=<us-east-1> | bedrock client created
DEBUG - strands.tools.registry - tools_dir=</workshop/strands_agents/01-first-agent/tools> | tools directory not found
DEBUG - strands.tools.registry - tool_modules=<[]> | discovered
DEBUG - strands.tools.registry - tool_count=<0>, success_count=<0> | finished loading tools
DEBUG - strands.tools.registry - getting tool configurations
DEBUG - strands.tools.registry - tool_count=<0> | tools configured
DEBUG - strands.tools.registry - getting tool configurations
DEBUG - strands.tools.registry - tool_count=<0> | tools configured
DEBUG - strands.event_loop.streaming - model=<<strands.models.bedrock.BedrockModel object at 0xffff43464b90>> | stream

Hello! It's nice to meet you. How are you doing today? Is there anything I can help you with?

DEBUG - strands.models.bedrock - finished streaming response from model
DEBUG - strands.agent.conversation_manager.sliding_window_conversation_manager - message_count=<2>, window_size=<40> | skipping context reduction


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Hello! It's nice to meet you. How are you doing today? Is there anything I can help you with?"}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[2.1112565994262695], traces=[<strands.telemetry.metrics.Trace object at 0xffff4305e950>], accumulated_usage={'inputTokens': 9, 'outputTokens': 27, 'totalTokens': 36}, accumulated_metrics={'latencyMs': 2074}), state={})


### 4. Model Provider

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.7 Sonnet in the region of your current AWS environment

You can specify a different model in Amazon Bedrock providing the model ID string directly:

In [6]:
from strands import Agent

agent = Agent(model="us.anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)

DEBUG - strands.models.bedrock - config=<{'model_id': 'us.anthropic.claude-3-5-haiku-20241022-v1:0', 'include_tool_result_status': 'auto'}> | initializing
INFO - botocore.credentials - Found credentials from IAM Role: code-editor-CodeEditorInstanceBootstrapRole-Irz22oEDgalJ
DEBUG - strands.models.bedrock - region=<us-east-1> | bedrock client created
DEBUG - strands.tools.registry - tools_dir=</workshop/strands_agents/01-first-agent/tools> | tools directory not found
DEBUG - strands.tools.registry - tool_modules=<[]> | discovered
DEBUG - strands.tools.registry - tool_count=<0>, success_count=<0> | finished loading tools


{'model_id': 'us.anthropic.claude-3-5-haiku-20241022-v1:0', 'include_tool_result_status': 'auto'}



For more control over the model configuration, you can create a `BedrockModel` provider instance:

In [7]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-east-1',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)

DEBUG - strands.models.bedrock - config=<{'model_id': 'us.anthropic.claude-3-5-haiku-20241022-v1:0', 'include_tool_result_status': 'auto', 'temperature': 0.3}> | initializing
INFO - botocore.credentials - Found credentials from IAM Role: code-editor-CodeEditorInstanceBootstrapRole-Irz22oEDgalJ
DEBUG - strands.models.bedrock - region=<us-east-1> | bedrock client created
DEBUG - strands.tools.registry - tools_dir=</workshop/strands_agents/01-first-agent/tools> | tools directory not found
DEBUG - strands.tools.registry - tool_modules=<[]> | discovered
DEBUG - strands.tools.registry - tool_count=<0>, success_count=<0> | finished loading tools



More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


**Congratulations !! Now you have learned how to build a simple agent using Strands!!**

## Lets Build a Task-Specific Agent - RecipeBot 🍽️

Lets create a practical example of a task-specific agent. We create a `RecipeBot` that recommends recipes and answers any cooking related questions. Lets dive in !!

Here's what we will create :

<div style="text-align:center">
    <img src="images/interactive_recipe_agent.png" width="75%" />
</div>

In [8]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [9]:
# Create a recipe assistant agent
recipe_agent = Agent(
    model="us.anthropic.claude-sonnet-4-20250514-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes based on ingredients and answer cooking questions.
    Use the websearch tool to find recipes when users mention ingredients or to
    look up cooking information.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

INFO - botocore.credentials - Found credentials from IAM Role: code-editor-CodeEditorInstanceBootstrapRole-Irz22oEDgalJ


In [10]:
response = call_with_retry(lambda: recipe_agent("Suggest a recipe with chicken and broccoli."))

# Other examples:
# response = call_with_retry(lambda: recipe_agent("How do I cook quinoa?"))
# response = call_with_retry(lambda: recipe_agent("How can I substitute white wine in shrimp pasta?"))
# response = call_with_retry(lambda: recipe_agent("What are the health benefits of asparagus?"))
print(response)

I'll search for some delicious chicken and broccoli recipes for you!
Tool #1: websearch


INFO - primp - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=chicken%20broccoli%20recipe%20easy 200
INFO - primp - response: https://grokipedia.com/api/typeahead?query=chicken+broccoli+recipe+easy&limit=1 200
INFO - primp - response: https://yandex.com/search/site/?text=chicken+broccoli+recipe+easy&web=1&searchid=3256146 200


Let me search for more specific recipe details:
Tool #2: websearch


INFO - primp - response: https://grokipedia.com/api/typeahead?query=%22chicken+and+broccoli+stir+fry%22+recipe+ingredients+instructions&limit=1 200
INFO - primp - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=%22chicken%20and%20broccoli%20stir%20fry%22%20recipe%20ingredients%20instructions 200
INFO - primp - response: https://yandex.com/search/site/?text=%22chicken+and+broccoli+stir+fry%22+recipe+ingredients+instructions&web=1&searchid=5818354 200


Based on my search, I found some great chicken and broccoli recipe options! Here's a delicious and easy **Chicken and Broccoli Stir-Fry** recipe that's perfect for a quick, healthy meal:

## Easy Chicken and Broccoli Stir-Fry

### Ingredients:
- 1 lb boneless, skinless chicken breast or thighs, cut into bite-sized pieces
- 4 cups fresh broccoli florets
- 2 tablespoons vegetable oil (divided)
- 3 cloves garlic, minced
- 1 tablespoon fresh ginger, minced
- 3 tablespoons soy sauce
- 2 tablespoons oyster sauce
- 1 tablespoon cornstarch
- 1 teaspoon sesame oil
- 1/4 cup chicken broth
- Salt and pepper to taste
- Cooked rice for serving

### Instructions:

1. **Prep first**: Cut chicken into bite-sized pieces and prepare all vegetables. This is key since stir-frying is fast!

2. **Make the sauce**: Mix soy sauce, oyster sauce, cornstarch, sesame oil, and chicken broth in a small bowl. Set aside.

3. **Cook the chicken**: Heat 1 tablespoon oil in a large skillet or wok over medium-high heat. 

#### And that's it! We now have a running usecase agent with tools in just a few lines of code 🥳.

For more detailed quickstart guide, check out the [Strands documentation](https://strandsagents.com/0.1.x/user-guide/quickstart/).

## Run RecipeBot via Streamlit Frontend: 

Now let's create an interactive web interface using the chatbot code you've built above!

### Step 1: Add Your Code

1. Open `chatbot_demo.py`
2. Copy the bedrock model initialization and agent creation code similar to that above:

   ```python
    boto3_session = boto3.session.Session()
    region = boto3_session.region_name or "us-east-1"
    
    # Configure logging
    logging.getLogger("strands").setLevel(logging.INFO)

    # Define a websearch tool
    @tool
    def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
        """Search the web to get updated information.
        Args:
            keywords (str): The search query keywords.
            region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
            max_results (int | None): The maximum number of results to return.
        Returns:
            List of dictionaries with search results.
        """
        try:
            results = DDGS().text(keywords, region=region, max_results=max_results)
            return results if results else "No results found."
        except RatelimitException:
            return "RatelimitException: Please try again after a short delay."
        except DDGSException as d:
            return f"DuckDuckGoSearchException: {d}"
        except Exception as e:
            return f"Exception: {e}"

    bedrock_model = BedrockModel(
        model_id=st.session_state.current_model,
        temperature=temperature,
        region_name=region
    )

    # Create a recipe assistant agent
    st.session_state.agent = Agent(
        model=bedrock_model,  # Optional: Specify the model ID
        system_prompt="""You are RecipeBot, a helpful cooking assistant.
        Help users find recipes based on ingredients and answer cooking questions.
        Use the websearch tool to find recipes when users mention ingredients or to
        look up cooking information.""",
        # Import the websearch tool we created above
        tools=[websearch],
    )

   ```
3. Paste the code into the designated section marked `# PASTE YOUR CODE HERE`

### Step 2: Run the Demo

1. Open a terminal
2. Change the directory to where the `chatbot_demo.py` file is:

    ```
    cd /workshop/strands_agents/01-first-agent
    ```
3. Run the script with streamlit:

    ```bash
    streamlit run chatbot_demo.py
    ```
4. You will see an output like:

    ```
    You can now view your Streamlit app in your browser.

    Local URL: http://localhost:8501
    ```
5. Go to the ports tab and open the Amazon CloudFront URL next to the port (here 8501) to access the streamlit application:

    ![The **Ports** tab in the Code Editor shows URLs to ports running on the remote server](./images/forwarded_port.png)

The web interface includes:
- Model selection
- Temperature control
- Chat history with automatic context preservation
- Clear chat functionality

### Additional Challenge
- Implement an id_generator tool using the ``@tool`` decorator. The tool takes an ID length as input and returns a 
random ID number. Don't forget to include parameters and tool description in the docstring.
- Add the tool to the agent you created and test it with diffrent prompts

In [11]:
""" ID_GENERATOR_TOOL  """

@tool
def id_generator(length: int = 9) -> str:
    """
    Generate a random ID number with specified length.
    
    Creates a random alphanumeric identifier that can be used for unique IDs,
    session tokens, or temporary identifiers. The generated ID contains both
    uppercase letters and digits for better uniqueness.
    
    Args:
        length (int): ...   
    """ 
    # COMPLETE CODE 